# Vertrauliche Vorhersagen mit Syft Keras

# Schritt 1: Öffentliches Training

Willkommen zu diesem Tutorial! In den folgenden Notebooks wird es darum gehen vertrauliche Vorhersagen anbieten zu können. Mit vertraulichen Vorhersagen ist gemeint, dass die Daten dabei über den gesamten Prozess hin verschlüsselt bleiben. Zu keinem Zeitpunkt teilt der Nutzer dabei Rohdaten. Um diese vertraulichen Vorhersagen anbieten zu können, nutzt Syft Keras eine zugrundeliegende Bibliothek names [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted). TF Encrypted kombiniert topaktuelle Kryptographie und Machine Learning Techniken. Dabei kann sich der Nutzer jedoch voll auf die Anwendung von Machine Learning konzentrieren.

Vertrauliche Vorhersagen anzubieten lässt sich in drei Schritten erreichen:
- **Schritt 1**: Trainiere das Model mit normalem Keras.
- **Schritt 2**: Sichere das Model ab und starte einen Server zum Anbieten.
- **Schritt 3**: Befrage das Model für eigene vertrauliche Vorhersagen als Klient.

Lassen Sie uns mit diesen drei Schritten beginnen, um wirklungsvolle Machine Learning Services ohne der Verlust der Privatsphäre anzubieten.

Autoren:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

Im Namen von:
- Dropout Labs - Twitter: [@dropoutlabs](https://twitter.com/dropoutlabs)
- TF Encrypted - Twitter: [@tf_encrypted](https://twitter.com/tf_encrypted)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

## Trainiere das Model mit Keras

Um vertrauliche Machine Learning Techniken für eigene Projekte anwenden zu können, sollte niemand neue Machine Learning Gerüste lernen müssen. Mit grundlegenden [Keras-Kenntnissen](https://keras.io/) können diese Techniken mit Syft Keras genutzt werden. Wenn Sie noch nie mit Keras gearbeitet haben, können Sie es mit der [Keras-Dokumentation](https://keras.io) besser kennenlernen. 

Bevor vertrauliche Vorhersagen angeboten werden können, muss erstmal ein Model mit normalem Keras trainiert werden. Als Beispiel wird hier ein Model zum klassifizieren handgeschriebener Ziffern verwendet. Um das Model zu trainierren wird der [MNIST-Datensatz](http://yann.lecun.com/exdb/mnist/) verwendet.

[Dises Beispiel](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) wird vom Keras Repository ausgeliehen. Um das Model zu trainieren, reicht es aus die Zell unten zu starten. 

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Speichere das Model mit seinen Gewichten für zukünftige Vorhersagen

Klasse! Das Model wurde trainiert. Mit `model.save()` werden die Gewichte des Models gespeichert. Im nächsten  Notebook werden diese Gewichte in Syft Keras geladen und ein Verhersage-Service gestartet.

In [ ]:
model.save('short-conv-mnist.h5')